# Gradient Based Optimization

## Gradient decent method    
The general framework for a gradient method for finding a minimizer of a function $f$ is defined by
\begin{align*} 
x^{k+1} = x^{(k)} -\alpha_k \nabla f(x_k),\ \ \ \ \ \ \  k=0,1,2,...
\end{align*} 
where $\alpha_k \gt 0$ is known as the step size or learning rate.    

In linear regression, the critical point has a closed-form solution. But that's not always the case especially for complicated functions. Thus, gradient decent is often used in machine learning.     

From notebook 01, the gradient (first order derivative) is
$$
\nabla_\beta L(\beta) = -2 X^T y + 2X^TX\beta,
$$

The weight vector can be iteratively updated by doing:
$$
\hat{\beta}^{(k+1)} = \hat{\beta}^{(k)} - \eta \nabla_{\beta} L(\hat{\beta}^{(k)}).
$$


First, generate some synthetic data:

In [2]:
import numpy as np
np.random.seed(8964)

n = 200                                                       # number of samples
# dimension of problem
p = 6
# noise standard deviation
sigma = 0.2
X = np.random.normal(0, 1, size=(n, p))                        # design matrix
beta_star = np.random.randint(-4, 2, p)                       # true beta
noise = np.random.normal(0, sigma, size=(n))
y = X @ beta_star + noise                                     # y values


#### Weight vector $\beta$ closed-form solution     
From notebook 01:
$$
\hat{\beta} = (X^TX)^{-1}X^T y.
$$


In [3]:
beta_hat_np = np.linalg.inv(X.T @ X) @ X.T @ y
print("beta hat using numpy:  ", beta_hat_np)
print("           true beta:  ", beta_star)


beta hat using numpy:   [-1.0202166  -2.99123646 -1.00450705 -1.97897081 -1.00582651 -2.01509766]
           true beta:   [-1 -3 -1 -2 -1 -2]


#### Solve with `sklearn`    

In [4]:
from sklearn.linear_model import LinearRegression
beta_hat_sk = LinearRegression(fit_intercept=False).fit(X, y).coef_
print("beta hat using sklearn:  ", beta_hat_sk)
print("             true beta:  ", beta_star)


beta hat using sklearn:   [-1.0202166  -2.99123646 -1.00450705 -1.97897081 -1.00582651 -2.01509766]
             true beta:   [-1 -3 -1 -2 -1 -2]


#### Gradient decent(GD)    
Take $\eta=0.001$ as the step size (learning rate), and run GD T=50 iterations.

As mentioned before, the gradient is
$$
\nabla_\beta L(\beta) = -2 X^T y + 2X^TX\beta,
$$

In [5]:
eta = 0.001
T = 50


def grad_loss(b, X, y):
    return -2 * X.T @ y + 2 * X.T @ X @ b


# store the beta vec at each iteration
betas = np.zeros(shape=(T, p))

for t in range(1, T):
    betas[t, :] = betas[t-1, :] - eta * grad_loss(betas[t-1, :], X, y)

print("beta hat using GD by hand:  ", betas[T-1, :])
print("                true beta:  ", beta_star)


beta hat using GD by hand:   [-1.02021658 -2.99123644 -1.00450708 -1.97897079 -1.00582652 -2.01509767]
                true beta:   [-1 -3 -1 -2 -1 -2]


#### GD with `PyTorch`    
PyTorch is technically an `automatic differentiation (autodiff)` library, which means that it is able to compute gradients of functions numerically. PyTorch is primarily used for deep learning (which requires a form of differentiation called 'back propagation'). 

In `PyTorch` tensors are used rather than `NumPy` arrays. You can think of tensors as you do arrays, the only difference is that tensors allow you to compute gradients. It is important to note here that in the following code, there is no equivalent of the `grad_loss` function, which is taken care of automatically (by the `autodiff`).

In [6]:
import torch
# X data tensor
X_tensor = torch.from_numpy(X).float()
# y data tensor
y_tensor = torch.from_numpy(y).float()
# this is our parameter vector beta
beta_tensor = torch.zeros(p, requires_grad=True)
eta = 0.1
T = 50

for _ in range(T):
    yhat = X_tensor @ beta_tensor             # prediction of model
    residual = yhat - y_tensor                # residual tensor (errors)
    loss = (residual**2).mean()               # mean squared error tensor

    # this call computes gradients of `loss` w.r.t. any params (beta)tensor
    loss.backward()

    with torch.no_grad():
        # we do not want this calculation to be part of the gradient computation
        beta_tensor -= eta * beta_tensor.grad

    # remove current gradients from being stored (important if you want to recompute gradients)
    beta_tensor.grad.zero_()

print("beta hat using PyTorch:  ", beta_tensor.data)
print("             true beta:  ", beta_star)


beta hat using PyTorch:   tensor([-1.0200, -2.9910, -1.0046, -1.9788, -1.0057, -2.0151])
             true beta:   [-1 -3 -1 -2 -1 -2]


#### GD with `PyTorch` a better way    
Make use of more `PyTorch` functionality.   

General procedures:
1. For any problem you wish to solve using `PyTorch`, you should first specify a `model` which inherits from the `Module` class and defines the model of computation you wish to work with. Importantly, this class always contains a `forward` function that describes how an input should make its way through the computational graph.

2. `PyTorch` already has a host of useful machine learning functions that can be imported. Many of these live in the `nn` module, which is short for `neural nets`.

3. `PyTorch` already has a large number of optimizers (other numerical methods apart from GD) that we can use. These live in the `optim` module

In [7]:
from torch import nn, optim

# create the linear regression model class


class LinearRegressionTorch(nn.Module):
    def __init__(self):
        super().__init__()

        # need to wrap the params of the model in nn.Parameter
        self.beta_tensor = nn.Parameter(torch.zeros(p, requires_grad=True))

    def forward(self, X):
        return X @ self.beta_tensor


model = LinearRegressionTorch()                        # create a model instance
loss_func = nn.MSELoss()                               # choose MSE loss
optimizer = optim.SGD(model.parameters(), lr=eta)      # choose Stochastic GD

for _ in range(50):
    yhat = model.forward(X_tensor)
    loss = loss_func(y_tensor, yhat)
    loss.backward()
    optimizer.step()                                   # no need to manually update
    optimizer.zero_grad()

print("beta hat using PyTorch w model:  ", model.beta_tensor.data)
print("                     true beta:  ", beta_star)


beta hat using PyTorch w model:   tensor([-1.0200, -2.9910, -1.0046, -1.9788, -1.0057, -2.0151])
                     true beta:   [-1 -3 -1 -2 -1 -2]


### Gradient based optimization on Ridge model     
Consider optimizing the following loss function of ridge model (w.r.t.  $x$):
$$
f(x)=\frac{1}{2}\|Ax-b\|_2^2+\frac{\gamma}{2}\|x\|^2_2
$$
and where $A\in \mathbb{R}^{m\times n}, b\in \mathbb{R}^m$ are defined as
$$
A=\left [
    \begin{array}{cccc}
    1&2&1&-1 \\
    -1&1&0&2 \\
    0&-1&-2&1 \\
    \end{array}
\right ]
,\ \ \ \ \ \ 
b = \left [
    \begin{array}{c}
    3\\ 2\\ -2\\
    \end{array}
\right ]
$$
$\gamma$ is a positive constant.    
Run gradient decent on $f$ with step size $\alpha = 0.1$, $\gamma = 0.2$, and starting point of $x^{(0)}=(1,1,1,1)$. 

Terminate the algorithm when $\|\nabla f(x^{(k)})\|_2 \lt 0.001$.

The gradient is 
\begin{align*}
\nabla f(x) &= \frac{\partial}{\partial x}f(x) = A^TAx-A^Tb+\gamma x
\end{align*}
So the GD steps are
\begin{align*}
x^{(k+1)} &= x^{(k)} - \alpha_k(A^TAx^{(k)}-A^Tb+\gamma x^{(k)}) \\ 
&= x^{(k)} - \alpha_k(A^T(Ax^{(k)}-b)+\gamma x^{(k)}) \\ 
\end{align*}


In [12]:
# Solve manually
A = np.array([
    [1, 2, 1, -1],
    [-1, 1, 0, 2],
    [0, -1, -2, 1]
])
b = np.array([3, 2, -2])
gamma = 0.2
alpha = 0.1
tol = 0.001


def grad_f(x):
    return A.T @ (A@x - b) + gamma * x


x = np.array([1, 1, 1, 1])
k = 0
x_k = [x]
while np.linalg.norm(grad_f(x)) > tol:
    x = x - alpha * grad_f(x)
    k += 1
    x_k.append(x)

print("First 5 iterations: ")
for i in range(6):
    print(f"k={i},\t{x_k[i]}")

print("Last 5 iterations: ")
for i in range(5):
    print(f"k={k - 4 + i},\t{x_k[k - 4 + i]}")


First 5 iterations: 
k=0,	[1 1 1 1]
k=1,	[0.98 0.98 0.98 0.98]
k=2,	[0.9624 0.9804 0.9744 0.9584]
k=3,	[0.942712 0.982392 0.966832 0.943272]
k=4,	[0.92337456 0.98663216 0.95983296 0.92954576]
k=5,	[0.90444937 0.99160416 0.95259323 0.91685286]
Last 5 iterations: 
k=272,	[0.06662967 1.33658172 0.49283021 0.32508074]
k=273,	[0.06655417 1.33661408 0.49278707 0.32502682]
k=274,	[0.06648018 1.33664579 0.49274479 0.32497397]
k=275,	[0.06640768 1.33667686 0.49270336 0.32492218]
k=276,	[0.06633662 1.33670732 0.49266275 0.32487142]


In [23]:
# Solve with PyTorch
from turtle import forward


A_tensor = torch.from_numpy(A).float()
b_tensor = torch.from_numpy(b).float()


class RidgeModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.x = nn.Parameter(torch.ones(4, requires_grad=True))

    def forward(self, A):
        return A @ self.x


model = RidgeModel()
optimizer = optim.SGD(model.parameters(), lr=alpha)

k = 0
while True:
    if k % 20 == 0:
        print(f"{k=},\t{model.x.data}")
    y = model.forward(A_tensor)
    loss = 0.5 * torch.linalg.norm(y - b_tensor, ord=2)**2 + \
        gamma * 0.5 * torch.linalg.norm(model.x, ord=2)**2
    loss.backward()
    optimizer.step()
    k += 1
    if torch.linalg.norm(model.x.grad) <= tol:
        break
    optimizer.zero_grad()

print(f"k={k},\t{model.x.data}")


k=0,	tensor([1., 1., 1., 1.])
k=20,	tensor([0.6772, 1.0758, 0.8405, 0.7608])
k=40,	tensor([0.4726, 1.1626, 0.7248, 0.6150])
k=60,	tensor([0.3364, 1.2210, 0.6470, 0.5177])
k=80,	tensor([0.2455, 1.2599, 0.5950, 0.4528])
k=100,	tensor([0.1848, 1.2860, 0.5603, 0.4095])
k=120,	tensor([0.1442, 1.3033, 0.5372, 0.3805])
k=140,	tensor([0.1172, 1.3149, 0.5217, 0.3612])
k=160,	tensor([0.0991, 1.3227, 0.5114, 0.3483])
k=180,	tensor([0.0871, 1.3278, 0.5045, 0.3397])
k=200,	tensor([0.0790, 1.3313, 0.4999, 0.3339])
k=220,	tensor([0.0736, 1.3336, 0.4968, 0.3301])
k=240,	tensor([0.0701, 1.3351, 0.4948, 0.3275])
k=260,	tensor([0.0677, 1.3361, 0.4934, 0.3258])
k=277,	tensor([0.0663, 1.3367, 0.4926, 0.3248])
